## 前言
体验了bedrock的sdk, knowledge base和langchain集成的接口，现在我们进行一个实践项目。

如今很多交易所要求上市公司出了财报以外，还需提交ESG报告，披露公司在环境、社会和公司治理等方面信息。2月8日，上海证券交易所、深圳证券交易所和北京证券交易所就《上市公司持续监管指引第X号——可持续发展报告（征求意见稿）》，向社会公开征求意见。该意见稿的发布意味着ESG信息的强制披露是未来趋势和要求。

ESG报告的撰写是高度专业性工作，内容广泛，要求复杂。而生成式AI的发展，很大程度上可以简化ESG报告的撰写。利用 RAG 技术结合企业私有ESG等数据，基于 LLM 和智能搜索，配合提示词工程，可为企业提供高质量的写作建议，甚至生成报告。接下来我们让我们一起体验Bedrock Knowledge Base和Claude3如何帮助公司生成ESG报告。

## 安装依赖

In [1]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q tqdm
!pip install -q langchain==0.1.11
!pip install -q langchain-core
!pip install -q langchain_community

In [ ]:
bucket_name = "请在此输入您的S3桶名"
!wget d13loqfldp8rtf.cloudfront.net/ESG.zip
!mkdir ESG
!unzip -d ./ESG ESG.zip
!aws s3 sync ./ESG/ s3://{bucket_name}

## Bedrock Knowledge Base 根据用户问题检索ESG报告相关内容

In [43]:
# 替换成您在之前实验构建的Knowledge Base的ID
kb_id = "your-bedrock-konwledgebase-id"

In [44]:
import boto3
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.chat_models import BedrockChat

## 请在此处输入您的问题
question = 'ZTO Express ESG polics and practices'

# 1. set up bedrock knowledge base client and retrieve
kb_client = boto3.client('bedrock-agent-runtime')
kb_response = kb_client.retrieve(
    knowledgeBaseId=kb_id,
    retrievalQuery={
        'text': question
    }
)
context = ""
for result in kb_response['retrievalResults']:
    context = context +'\n'+ result['content']['text']
    print(result['location'])
    print(result['content']['text'])
    print("----------------")

{'type': 'S3', 's3Location': {'uri': 's3://knowledgebase-bedrock-agent-1223a/ESG报告_中文_中通快递_T500c.com_2020_41P.pdf'}}
While enhancing the understanding and  support of stakeholders for the Company, the Company also accepts reasonable opinions and suggestions from internal and  external stakeholders and continues to improve the business management method and enhance the Company's comprehensive  competitiveness.    ZTO Express’s responsibility governance framework is based on good corporate governance and lofty ethical  standards. Economic, social and environmental factors are comprehensively considered in the business  decision-making process at all levels. The Board of Directors (BoD) assumes full responsibility for the  Company’s ESG strategies and reports and discusses sustainable development issues at its regular meetings.  The BoD also established a comprehensive management system involving management participation as  well as the horizontal coordination of multiple departments so a

## Claude3 基于检索到的文本生成回答

In [45]:
# 2. set up llm - claude3
claude3_kwargs = {
    "temperature":0.0001,
    #"temperature":1,
    "top_k":250,
    "top_p":0.999,
    "max_tokens": 3000,
}
claude3 = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=claude3_kwargs
)
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_01 = LLMChain(llm=claude3, prompt=prompt)

response_01=chain_01.invoke({'question': question,'context': context})
print(response_01['text'])

Based on the context provided, here are some key points about ZTO Express's ESG policies and practices:

- ZTO Express has an ESG and CSR governance framework overseen by the Board of Directors, which assumes responsibility for the company's ESG strategies and discusses sustainable development issues regularly.

- They have established an ESG work management team led by the capital market department and a CSR work management team led by the brand department to manage, promote and report on ESG and CSR initiatives.

- The company emphasizes compliance and has obtained certifications like ISO 9001, ISO 27001, ISO 27701 related to quality, information security, privacy management.

- ZTO Express actively discloses information to investors through channels like investor relations department, roadshows, website to protect investor rights.

- In 2021, the company joined the Science Based Targets initiative (SBTi) and committed to setting science-based carbon emission reduction targets aligne

## Claude3 针对某个ESG议题生成报告

In [46]:
prompt_template_02 = """
你是撰写ESG（环境、社会和治理）报告的专家。

本次,我们需要您为一家{industry}行业的公司,就 GRI 重要性议题"{topic}"编写 ESG 报告内容。

公司提供的相关数据:
{statistics}

此外,您将参考其他公司的 ESG 信息:
{relevant_info}

报告内容要求:
1. 字数在 5000 字以上
2. 遵循 GRI 标准框架的要求
3. 阐述公司在该议题下的管理政策、具体行动及未来规划
4. 根据数据给出一些详细的实际案例
5. 报告结构合理,行文条理清晰,信息连贯一致

请按照上述要求,为我们撰写高质量的 ESG 报告内容。
"""

prompt_02 = PromptTemplate(
    template=prompt_template_02, input_variables=["industry","topic","statistics","relevant_info"]
)

claude3_kwargs = {
    "temperature":1,
    "top_k":250,
    "top_p":1,
    "max_tokens": 3000,
}
claude3 = BedrockChat(
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",
    model_kwargs=claude3_kwargs
)
chain_02 = LLMChain(llm=claude3, prompt=prompt_02, verbose=True)

result = chain_02.invoke({
    "industry": "物流",
    "topic": "水资源和污水",
    "statistics":'''2022年废水排放量为 2,00.00 万 m3，废水排放强度为 2.22m3/ 万元，中水回用量为 66.66 万 m3。''',
    "relevant_info": response_01['text']
})

print(result['text'])



> Entering new LLMChain chain...
Prompt after formatting:

你是撰写ESG（环境、社会和治理）报告的专家。

本次,我们需要您为一家物流行业的公司,就 GRI 重要性议题"水资源和污水"编写 ESG 报告内容。

公司提供的相关数据:
2022年废水排放量为 2,00.00 万 m3，废水排放强度为 2.22m3/ 万元，中水回用量为 66.66 万 m3。

此外,您将参考其他公司的 ESG 信息:
Based on the context provided, here are some key points about ZTO Express's ESG policies and practices:

- ZTO Express has an ESG and CSR governance framework overseen by the Board of Directors, which assumes responsibility for the company's ESG strategies and discusses sustainable development issues regularly.

- They have established an ESG work management team led by the capital market department and a CSR work management team led by the brand department to manage, promote and report on ESG and CSR initiatives.

- The company emphasizes compliance and has obtained certifications like ISO 9001, ISO 27001, ISO 27701 related to quality, information security, privacy management.

- ZTO Express actively discloses information to investors through channels lik